# **INTRODUCTIONS**

* * *

The source code is separated into multiple sections:

1. **[Tables](#tables)**
3. **[Stored Procedures](#stored-procedures)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/statics/sewers/inactive_sewer_installation_mapping.ipynb",
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**

---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS](
  [ID] [INT] PRIMARY KEY IDENTITY(1,1),
  [VERSION] [VARCHAR](100) NULL,
  [DATE_MEASUREMENT] [VARCHAR](100) NULL,
  [RWZI_AWZI_CODE] [VARCHAR](100) NULL,
  [RWZI_AWZI_NAME] [VARCHAR](100) NULL,
  [RNA_PER_ML] [VARCHAR](100) NULL,
  [RNA_FLOW_PER_100000] [VARCHAR](100) NULL,
  [DATE_OF_REPORT] [VARCHAR](100) NULL,
  [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO


### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[RIVM_SEWER_MEASUREMENTS]'))
CREATE TABLE [VWSINTER].[RIVM_SEWER_MEASUREMENTS](
    [ID] [INT] PRIMARY KEY IDENTITY(1,1),
    [DATE_MEASUREMENT] [DATETIME],
    [VERSION] [VARCHAR](100) NULL,
    [RWZI_AWZI_CODE] [BIGINT],
    [RWZI_AWZI_NAME] [VARCHAR](100) NULL,
    [RNA_PER_ML] [BIGINT],
    [RNA_FLOW_PER_100000] [BIGINT],
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE()
);
GO

# **STORED PROCEDURES**

---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT. 
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- MOVE SEWER MEASUREMENTS DATA FROM STAGING TO INTERMEDIATE TABLE.

-- COMMENTS:
-- 1. EITHER RNA_PER_ML OR RNA_FLOW_PER_100000 IS FILLED. THE OTHER IS EMPTY: RNA_PER_ML SEEMS LEGACY
-- 2. RNA_FLOW_PER_100000 CAN BE EITHER EMPTY OR 0. RECORDS WITH NO VALUES ARE LEFT OUT (RNA_PER_ML WAS MEASURED)

CREATE OR ALTER PROCEDURE [DBO].[SP_RIVM_SEWER_MEASUREMENTS_INTER]
AS
BEGIN
    INSERT INTO [VWSINTER].[RIVM_SEWER_MEASUREMENTS] (
        T1.[DATE_MEASUREMENT]
        ,T1.[VERSION]
        ,T1.[RWZI_AWZI_CODE]
        ,T1.[RWZI_AWZI_NAME]
        ,T1.[RNA_PER_ML]
        ,T1.[RNA_FLOW_PER_100000]
    )
    SELECT 
        CONVERT(DATETIME, T1.[DATE_MEASUREMENT], 126) AS [DATE_MEASUREMENT]
        ,T1.[VERSION]
        ,T1.[RWZI_AWZI_CODE]
        ,T1.[RWZI_AWZI_NAME]
        ,CAST((SELECT CONVERT(FLOAT, ISNULL(NULLIF(T1.[RNA_PER_ML], ''), '0'))) AS BIGINT) AS [RNA_PER_ML]
        ,CONVERT(BIGINT, CONVERT(FLOAT, REPLACE(T1.[RNA_FLOW_PER_100000],',','.'))) AS [RNA_FLOW_PER_100000]
    FROM 
       [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS] T1
    LEFT JOIN [VWSSTATIC].[INACTIVE_SEWER_INSTALLATION_MAPPING] T2
        ON CAST(T1.[RWZI_AWZI_CODE] AS INT) = T2.[RWZI_CODE]
    WHERE T1.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS])
        AND T1.[RNA_FLOW_PER_100000] != ''
        AND ISNULL(T2.[ACTIVE], 1) != 0

    TRUNCATE TABLE [VWSSTAGE].[RIVM_SEWER_MEASUREMENTS]
END;
GO